#Some changes were done to perception.py
1) added code to update the world map only when the rover is flat on the ground aka (pitch and roll close to or equal 0 or 360)

In [ ]:
if (Rover.pitch < 1 or Rover.pitch > 359) and (Rover.roll < 1 or Rover.roll > 359):
        Rover.worldmap[o_y, o_x, 0] = 255
        Rover.worldmap[rock_y, rock_x,1] = 255
        Rover.worldmap[navigable_y, navigable_x, 2] = 255
        # remove overlap mesurements
        nav_pix = Rover.worldmap[:, :, 2] > 0
        Rover.worldmap[nav_pix, 0] = 0
        # clip to avoid overflow
        Rover.worldmap = np.clip(Rover.worldmap, 0, 255)
        Rover.worldmap[o_y, o_x, 0] += 1
        Rover.worldmap[rock_y, rock_x, 1] += 1
        Rover.worldmap[navigable_y, navigable_x, 2] += 1
    return Rover

2)added the polar angles and distances of the rock sample

In [ ]:
x1,y1 = rover_coords(rock_samples)
dst, angles = to_polar_coords(x1, y1)

3) added limit_range function to limit the range of the rovers camera in an attempt to improve fidelity

In [ ]:
def limit_range(x,y,range = 80):
    dist= np.sqrt(x**2 + y**2)
    return x[dist < range], y[dist < range]

In decision step we mostly overhauled the code and added different elements to help the rover navigate and find all the rock samples
1) We moved the rock pick up condition to be the first thing checked in the decision step

In [ ]:
if Rover.near_sample and Rover.vel == 0 and not Rover.picking_up:
        Rover.send_pickup = True
        return Rover

2) Next we check whether we are near the sample or not in order to send a pick up command

In [ ]:
if Rover.near_sample:
        Rover.mode = 'stop'
        Rover.throttle = 0
        Rover.brake = Rover.brake_set
        # Rover.send_pickup = True
        return Rover

3) next we check if we are stuck in a circle then the rover moves forward if it is stuck in a circle

In [ ]:
if Rover.mode == "stuckCircle":
        if time.time() - circle_start_time > 15:
            Rover.mode = "forward"
            circle_start_time=time.time()
        else:
            Rover.throttle = 0
            Rover.brake = 0
            Rover.steer = -10
        return Rover

4) we added some code to detect if we pass the rock (this code shouldnt execute since we almost always pick up the rock)

In [ ]:
 #Did we overshoot the pickup?
if Rover.picking_up:
    if not Rover.near_sample:
        Rover.mode='stop'
        Rover.picking_up=0
        Rover.send_pickup = False
        Rover.steer=-15
    return Rover

5) The following lines of code are very vital for picking up the rocks

* it detects if there are rocks in the rovers field of view
* then it stops immediately and proceeds towards the rock slowly

In [ ]:
#Steer to direction of the sample
if len(Rover.samples_angles) > 1:
        Rover.brake = Rover.brake_set
        Rover.mode = 'stop'
        Rover.steer = np.clip(np.mean(Rover.samples_angles * 180/np.pi), -15, 15)
        if (np.mean(Rover.samples_dists) >= 4):
            Rover.throttle = 0.4
            Rover.brake = 0
        elif (np.mean(Rover.samples_dists) <= 4):
            Rover.brake = Rover.brake_set
            Rover.near_sample = True
        return Rover

5) This code is implemented twice once at the beggining of the decision step and once again in the stop state

In [ ]:
elif Rover.mode == 'stop':
    # If we're in stop mode but still moving keep braking
    if len(Rover.samples_angles) > 1:
        Rover.brake = Rover.brake_set
        Rover.steer = np.clip(np.mean(Rover.samples_angles * 180/np.pi), -15, 15)
        if (np.mean(Rover.samples_dists) >= 3):
            Rover.throttle = 0.2
            Rover.brake = 0
        elif (np.mean(Rover.samples_dists) <= 3):
            Rover.brake = Rover.brake_set
            Rover.near_sample = True
        return Rover

6) we have the forward state
* First thing we do is we check if the rover is stuck in forward mode but not moving

In [ ]:
# #check if STUCK under a rock but can see navigable terrain
if Rover.vel <= 0.1 and Rover.throttle!=0 and len(Rover.samples_angles) == 0:
    if time.time() - stuck_start_time > 1.5:
        Rover = unstuck_action(Rover)
        return Rover
    else:
                stuck_start_time = time.time()

* we check for navigable terrain and apply a throttle in the direction of the mean of the navigation angles

In [ ]:
# Check the extent of navigable terrain
if len(Rover.nav_angles) >= Rover.stop_forward:  
    # If mode is forward, navigable terrain looks good 
    # and velocity is below max, then throttle 
    if Rover.vel < Rover.max_vel:
        # Set throttle value to throttle setting
        Rover.throttle = Rover.throttle_set
    else: # Else coast
        Rover.throttle = 0
    Rover.brake = 0
    # Set steering to average angle clipped to the range +/- 15
    try:
        Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi) + offset, -15, 15)
    except:
        print("mean of nav angles caught an exception")

* otherwise if theres a lack of navigable terrain we switch to stop mode where we navigate to the right (-15) in order to find navigable terrain

In [ ]:
# If there's a lack of navigable terrain pixels then go to 'stop' mode
elif len(Rover.nav_angles) < Rover.stop_forward:
        # Set mode to "stop" and hit the brakes!
        Rover.throttle = 0
        # Set brake to stored brake value
        Rover.brake = Rover.brake_set
        Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi)+ offset, -15, 15)
        Rover.mode = 'stop'

7) Stop mode
* as discussed before the first thing we do in the stop state is check whether there are rock samples or not then if there are we immediately break then steer towards the samples

In [1]:
 # If we're already in "stop" mode then make different decisions
elif Rover.mode == 'stop':
    # If we're in stop mode but still moving keep braking
    if len(Rover.samples_angles) > 1:
        Rover.brake = Rover.brake_set
        Rover.steer = np.clip(np.mean(Rover.samples_angles * 180/np.pi), -15, 15)
        if (np.mean(Rover.samples_dists) >= 3):
            Rover.throttle = 0.2
            Rover.brake = 0
        elif (np.mean(Rover.samples_dists) <= 3):
            Rover.brake = Rover.brake_set
            Rover.near_sample = True
        return Rover

SyntaxError: invalid syntax (2505194644.py, line 2)

8) (still in stop mode) we then check to se if we havent stopped completely then set the brakes and stop 

In [ ]:
if Rover.vel > 0.2:
    Rover.throttle = 0
    Rover.brake = Rover.brake_set
    Rover.steer = 0

9) then we check if the velocity is below 0.2 and there isnt enough navigable terrain then we simply steer to the right (because we are hugging the left wall) otherwise if there is enough navigable terrain we move in the direction of the mean of the angles of the navigable terrain

In [ ]:
elif Rover.vel <= 0.2:
    # Now we're stopped and we have vision data to see if there's a path forward
    if (len(Rover.nav_angles) < Rover.go_forward) and len(Rover.samples_angles) == 0:
        Rover.throttle = 0
        # Release the brake to allow turning
        Rover.brake = 0
        # Turn range is +/- 15 degrees, when stopped the next line will induce 4-wheel turning
        Rover.steer = -15 # Could be more clever here about which way to turn
    # If we're stopped but see sufficient navigable terrain in front then go!
    if len(Rover.nav_angles) >= Rover.go_forward:
        # Set throttle back to stored value
        Rover.throttle = Rover.throttle_set
        # Release the brake
        Rover.brake = 0
        # Set steer to mean angle
        try:
            Rover.steer = np.clip(np.mean(Rover.nav_angles  * 180/np.pi) + offset, -15, 15)
        except:
            print("mean of nav angles caught an exception")
        Rover.mode = 'forward'

10) finally we check if the rover is stuck in a circle because there are no navigable terrain then we start moving forward as soon as we see little navigable terrain (20)

In [ ]:
if Rover.vel == 0 and Rover.steer == -15 and len(Rover.nav_angles) > 20 and len(Rover.samples_angles) == 0 :
                Rover.steer = 0
                Rover.throttle = 0.5
                Rover.mode = 'forward'